# Plotting the predictions of the single- and multi-center RPN signatures
- This jupyter notebook is available on-line at:
  - https://github.com/spisakt/RPN-signature/blob/master/notebooks/3_compare_predictions.ipynb
- Input data for the notebook and non-standard code (PAINTeR library) is available in the repo:
  - https://github.com/spisakt/RPN-signature
- Raw MRI-data from study-centers 1 and 2 are available on OpenNeuro:
  - https://openneuro.org/datasets/ds002608/versions/1.0.1
  - https://openneuro.org/datasets/ds002609/versions/1.0.3
- Raw data from center 3 is available upon reasonable request.

## Imports

In [2]:
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn.connectome import vec_to_sym_matrix, sym_matrix_to_vec
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
from mlxtend.evaluate import permutation_test
from scipy.stats import f_oneway
from scipy import stats

import sys
sys.path.append('../')
from PAINTeR import plot # in-house lib used for the RPN-signature

from tqdm import tqdm

AttributeError: type object 'statsmodels.tsa.statespace._kalman_smoother.array' has no attribute '__reduce_cython__'

## Load and merge behavioral data for all three centers (after exclusions)

In [ ]:
df_bochum = pd.read_csv("../res/bochum_sample_excl.csv")
df_essen = pd.read_csv("../res/essen_sample_excl.csv")
df_szeged = pd.read_csv("../res/szeged_sample_excl.csv")
df_bochum['study']='bochum'
df_essen['study']='essen'
df_szeged['study']='szeged'
df=pd.concat((df_bochum, df_essen, df_szeged), sort=False)
df=df.reset_index()
y = df.mean_QST_pain_sensitivity

## Load predictions for the original single-center and the newly proposed multi-center model

In [ ]:
# predictions
multicener_predictions = np.genfromtxt('../res/multi-center/nested_cv_combat_conservative_pred_full_GroupKFold30.csv', delimiter=',')
rpn_predictions = np.hstack((df_bochum.nested_prediction,
                            df_essen.prediction,
                            df_szeged.prediction))

predictions = {
    'single-center' : rpn_predictions,
    'multi-center' : multicener_predictions
}

### create study masks

In [ ]:
study_masks = {
    "study 1" : (df.study == 'bochum').values,
    "study 2" : (df.study == 'essen').values,
    "study 3" : (df.study == 'szeged').values,
    "study 1+2+3" : np.array([True] * len(y))
}

## Observed vs. Predicted plots

In [ ]:
sns.set_style('ticks')
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(10,6), sharex=True, sharey=True)

cols = ['tab:blue', 'tab:orange']

for row, cv in enumerate(predictions.keys()):
    for col, study in enumerate(study_masks.keys()):
        g=sns.regplot(y[study_masks[study]], predictions[cv][study_masks[study]], ax=axs[row, col],
                    scatter=True, scatter_kws={'alpha':0.3}, color=cols[row])
        g.set(xlabel=None) 
        axs[row, col].set_xlim([-2, 2])
        axs[row, col].set_ylim([-1.2, 1.2])
        axs[row, col].spines['top'].set_visible(False)
        axs[row, col].spines['bottom'].set_visible(False)
        axs[row, col].spines['right'].set_visible(False)
        axs[row, col].spines['left'].set_visible(False)
        axs[row, col].grid(True)
        
        print('***', cv, study, '****************************************************')
                   
        corr = np.corrcoef(y[study_masks[study]], predictions[cv][study_masks[study]])[0,1]
        axs[row, col].title.set_text("R={:.2f}".format(corr))
        print("R={:.2f}".format(corr))
        
        # takes some seconds
        p_corr = permutation_test(y[study_masks[study]], predictions[cv][study_masks[study]],
                           func=lambda x, y: np.corrcoef(x, y)[0,1],
                           method='approximate',
                           num_rounds=8000,
                           seed=42)
        print("p_corr={:.5f}".format(p_corr))
        
        mse = mean_squared_error(y[study_masks[study]], predictions[cv][study_masks[study]])
        print("MSE={:.2f}".format(mse))
        
        mae = mean_absolute_error(y[study_masks[study]], predictions[cv][study_masks[study]])
        print("MSE={:.2f}".format(mae))
        
        expvar = explained_variance_score(y[study_masks[study]], predictions[cv][study_masks[study]])
        print("Expl. Var. ={:.3f}".format(expvar))
   
   
        
plt.savefig('../res/multi-center/regplots_obs-pred_combat.pdf')  

## Violoin plots per center for the observed and predicted values

In [ ]:
sns.violinplot(data=df, x='study', y=predictions['single-center'])

In [ ]:
sns.violinplot(data=df, x='study', y=predictions['multi-center'])

In [ ]:
sns.violinplot(data=df, x='study', y='mean_QST_pain_sensitivity')

In [ ]:
print(df.mean_QST_pain_sensitivity[df.study=='bochum'].mean())
print(df.mean_QST_pain_sensitivity[df.study=='essen'].mean())
print(df.mean_QST_pain_sensitivity[df.study=='szeged'].mean())

f_oneway(df.mean_QST_pain_sensitivity[df.study=='bochum'],
        df.mean_QST_pain_sensitivity[df.study=='essen'],
        df.mean_QST_pain_sensitivity[df.study=='szeged'])

## Remove main site effect from observed pain sensitivity and re-calculate measures

In [ ]:
y_res = np.copy(y.values)
y_res[df.study=='bochum'] -= np.mean(y_res[df.study=='bochum'])
y_res[df.study=='essen'] -= np.mean(y_res[df.study=='essen'])
y_res[df.study=='szeged'] -= np.mean(y_res[df.study=='szeged'])

df['mean_QST_pain_sensitivity_residual'] = y_res
sns.violinplot(data=df, x='study', y='mean_QST_pain_sensitivity_residual')

In [ ]:
sns.set_style('ticks')
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(10,6), sharex=True, sharey=True)

cols = ['tab:blue', 'tab:orange']

for row, cv in enumerate(predictions.keys()):
    for col, study in enumerate(study_masks.keys()):
        g=sns.regplot(y_res[study_masks[study]], predictions[cv][study_masks[study]], ax=axs[row, col],
                    scatter=True, scatter_kws={'alpha':0.3}, color=cols[row])
        g.set(xlabel=None) 
        axs[row, col].set_xlim([-2, 2])
        axs[row, col].set_ylim([-1.2, 1.2])
        axs[row, col].spines['top'].set_visible(False)
        axs[row, col].spines['bottom'].set_visible(False)
        axs[row, col].spines['right'].set_visible(False)
        axs[row, col].spines['left'].set_visible(False)
        axs[row, col].grid(True)
        
        print('***', cv, study, '****************************************************')
                   
        corr = np.corrcoef(y_res[study_masks[study]], predictions[cv][study_masks[study]])[0,1]
        axs[row, col].title.set_text("R={:.2f}".format(corr))
        print("R={:.2f}".format(corr))
        
        # takes some seconds
        p_corr = permutation_test(y_res[study_masks[study]], predictions[cv][study_masks[study]],
                           func=lambda x, y: np.corrcoef(x, y)[0,1],
                           method='approximate',
                           num_rounds=8000,
                           seed=42)
        print("p_corr={:.5f}".format(p_corr))
        
        mse = mean_squared_error(y_res[study_masks[study]], predictions[cv][study_masks[study]])
        print("MSE={:.2f}".format(mse))
        
        mae = mean_absolute_error(y_res[study_masks[study]], predictions[cv][study_masks[study]])
        print("MSE={:.2f}".format(mae))
        
        expvar = explained_variance_score(y_res[study_masks[study]], predictions[cv][study_masks[study]])
        print("Expl. Var. ={:.3f}".format(expvar))
   
   
        
plt.savefig('../res/multi-center/regplots_obs_resid-pred.pdf')  

## Expected correlation with study

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import importlib
import contextlib
import joblib
from joblib import Parallel, delayed

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """
    Context manager to patch joblib to report into tqdm progress bar given as argument
    Based on: https://stackoverflow.com/questions/37804279/how-can-we-use-tqdm-in-a-parallel-execution-with-joblib
    """
    def tqdm_print_progress(self):
        if self.n_completed_tasks > tqdm_object.n:
            n_completed = self.n_completed_tasks - tqdm_object.n
            tqdm_object.update(n=n_completed)

    original_print_progress = joblib.parallel.Parallel.print_progress
    joblib.parallel.Parallel.print_progress = tqdm_print_progress

    try:
        yield tqdm_object
    finally:
        joblib.parallel.Parallel.print_progress = original_print_progress
        tqdm_object.close()
#r_obs_pred = np.corrcoef(df.mean_QST_pain_sensitivity, predictions['multi-center'])[0,1]

data = pd.DataFrame(
    {
        'observed': df.mean_QST_pain_sensitivity.values,
        'predicted': predictions['multi-center'],
        'confounder': df.study.astype("category").cat.codes.values
    })

fit=ols("observed ~ predicted", data=data).fit()
print("R^2_(observed ~ predicted) =", fit.rsquared)

fit=ols("observed ~ C(confounder)", data=data).fit()
print("R^2_(observed ~ C(confounder)) =", fit.rsquared)
r2_obs_conf_true = fit.rsquared

fit=ols("predicted ~ C(confounder)", data=data).fit()
print("R^2_(predicted ~ C(confounder) =", fit.rsquared)
r2_pred_conf_true = fit.rsquared


corrs=[]
nulldata=[]

tolerance = 0.05



def workhorse(i):
    rng = np.random.default_rng(42+i)
    data_rs = pd.DataFrame(
        {
        'observed': df.mean_QST_pain_sensitivity.values,
        'predicted': predictions['multi-center'],
        'confounder': rng.permutation(df.study.astype("category").cat.codes.values)
        })
    r2_obs_conf_rs = ols("observed ~ C(confounder)", data=data_rs).fit().rsquared
    
    if (np.abs(r2_obs_conf_rs - r2_obs_conf_true) < tolerance):
        #print("Resampled R^2_(observed ~ C(confounder)) =", r2_obs_conf_rs)
        fit=ols("predicted ~ C(confounder)", data=data_rs).fit()
        #print("Resampled  R^2_(predicted ~ C(confounder) =", fit.rsquared)     
        return fit.rsquared, r2_obs_conf_rs
    return np.nan, np.nan
    
num_perms=100000  
with tqdm_joblib(tqdm(desc='permuting', total=num_perms)) as progress_bar:
            res = Parallel(n_jobs=-1)(delayed(workhorse)(i) for i in np.arange(num_perms))
    

nulldata, corrs = zip(*res)
nulldata = np.array(nulldata)
nulldata = nulldata[~np.isnan(nulldata)]
print(len(nulldata))
sns.distplot(nulldata)
plt.axvline(r2_pred_conf_true)

print("p=", np.sum(nulldata>=r2_pred_conf_true)/len(nulldata))

In [ ]:
corrs = np.array(corrs)
sns.distplot(corrs[~np.isnan(corrs)])
plt.axvline(r2_pred_conf_true)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import importlib
import contextlib
import joblib
from joblib import Parallel, delayed

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """
    Context manager to patch joblib to report into tqdm progress bar given as argument
    Based on: https://stackoverflow.com/questions/37804279/how-can-we-use-tqdm-in-a-parallel-execution-with-joblib
    """
    def tqdm_print_progress(self):
        if self.n_completed_tasks > tqdm_object.n:
            n_completed = self.n_completed_tasks - tqdm_object.n
            tqdm_object.update(n=n_completed)

    original_print_progress = joblib.parallel.Parallel.print_progress
    joblib.parallel.Parallel.print_progress = tqdm_print_progress

    try:
        yield tqdm_object
    finally:
        joblib.parallel.Parallel.print_progress = original_print_progress
        tqdm_object.close()
#r_obs_pred = np.corrcoef(df.mean_QST_pain_sensitivity, predictions['multi-center'])[0,1]

data = pd.DataFrame(
    {
        'observed': df.mean_QST_pain_sensitivity.values,
        'predicted': predictions['multi-center'],
        'confounder': df.study.astype("category").cat.codes.values
    })

fit=ols("observed ~ predicted", data=data).fit()
print("R^2_(observed ~ predicted) =", fit.rsquared)

fit=ols("observed ~ C(confounder)", data=data).fit()
print("R^2_(observed ~ C(confounder)) =", fit.rsquared)
r2_obs_conf_true = fit.rsquared

fit=ols("predicted ~ C(confounder)", data=data).fit()
print("R^2_(predicted ~ C(confounder) =", fit.rsquared)
r2_pred_conf_true = fit.rsquared


    corrs=[]
    nulldata=[]

    tolerance = 0.001



    def workhorse(i):
        rng = np.random.default_rng(4242+i)
        data_rs = pd.DataFrame(
            {
            'observed': df.mean_QST_pain_sensitivity.values,
            'predicted': predictions['multi-center'],
            'confounder': rng.permutation(df.study.astype("category").cat.codes.values)
            })
        r2_obs_conf_rs = ols("observed ~ C(confounder)", data=data_rs).fit().rsquared

        if (np.abs(r2_obs_conf_rs - r2_obs_conf_true) < tolerance):
            #print("Resampled R^2_(observed ~ C(confounder)) =", r2_obs_conf_rs)
            fit=ols("predicted ~ C(confounder)", data=data_rs).fit()
            #print("Resampled  R^2_(predicted ~ C(confounder) =", fit.rsquared)     
            return fit.rsquared, r2_obs_conf_rs
        return np.nan, np.nan

    num_perms=1000000 
    with tqdm_joblib(tqdm(desc='permuting', total=num_perms)) as progress_bar:
                res = Parallel(n_jobs=-1)(delayed(workhorse)(i) for i in np.arange(num_perms))


    nulldata, corrs = zip(*res)
    nulldata = np.array(nulldata)
    nulldata = nulldata[~np.isnan(nulldata)]
    print(len(nulldata))
    sns.distplot(nulldata)
    plt.axvline(r2_pred_conf_true)

    print("p=", np.sum(nulldata>=r2_pred_conf_true)/len(nulldata))

In [ ]:
corrs = np.array(corrs)
sns.distplot(corrs[~np.isnan(corrs)])
plt.axvline(r2_obs_conf_true)

In [ ]:
plt.figure(figsize=(8, 3))

print((np.max(nulldata)-np.min(nulldata))/10)

g=sns.distplot(nulldata, norm_hist=True, bins=10)
plt.axvline(r2_pred_conf_true, c='red')
g.set(xlabel=None) 
g.spines['top'].set_visible(False)
g.spines['bottom'].set_visible(False)
g.spines['right'].set_visible(False)
g.spines['left'].set_visible(False)
g.grid(True)
plt.savefig('../res/multi-center/hist_center_bias.pdf')  

In [ ]:
from scipy.stats import beta

def binom_interval(success, total, confint=0.95):
    quantile = (1 - confint) / 2.
    lower = beta.ppf(quantile, success, total - success + 1)
    upper = beta.ppf(1 - quantile, success + 1, total - success)
    return (lower, upper)

binom_interval(256*0.6484375, 256)

## Predicted vs. Predicted plot

In [ ]:
g=sns.jointplot(predictions['single-center'], predictions['multi-center'], kind='reg', color='black', scatter = False )
g.ax_joint.scatter(predictions['single-center'],predictions['multi-center'], c=df.mean_QST_pain_sensitivity,
                   cmap="coolwarm")
g.fig.set_size_inches(6,6)
g.ax_joint.set(xlabel=None) 
g.ax_joint.set_xlim([-2, 2])
g.ax_joint.set_ylim([-1.2, 1.2])
g.ax_joint.spines['top'].set_visible(False)
g.ax_joint.spines['bottom'].set_visible(False)
g.ax_joint.spines['right'].set_visible(False)
g.ax_joint.spines['left'].set_visible(False)
g.ax_joint.grid(True)
plt.savefig('../res/multi-center/regplots_pred-pred_combat.pdf')  

In [ ]:
norm = plt.Normalize(df.mean_QST_pain_sensitivity.min(), df.mean_QST_pain_sensitivity.max())
sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
plt.colorbar(sm)
plt.savefig('../res/multi-center/regplots_pred-pred_colorbar.pdf')  

In [ ]:
corr = np.corrcoef(predictions['single-center'], predictions['multi-center'])[0,1]
print("R={:.2f}".format(corr))
        
# takes some seconds
p_corr = permutation_test(y[study_masks[study]], predictions[cv][study_masks[study]],
                           func=lambda x, y: np.corrcoef(x, y)[0,1],
                           method='approximate',
                           num_rounds=8000,
                           seed=42)
print("p_corr={:.5f}".format(p_corr))